In [1]:
# configuration parameters
class Config:
    def __init__(self):
        self.POPULATION_SIZE = 1 # 150
        self.INPUTS = 4
        self.OUTPUTS = 2
        
        self.MUTATE_WEIGHTS_PROPORTION = 0.75

CONFIG = Config()

In [2]:
# TODO: incoming_edge can be a list of innovation numbers

NEURON_TYPES = ["input", "output", "hidden"]

class Neuron:
    NEURON_ID = 0
    
    def __init__(self, neuron_type = "hidden", incoming_edges = [], value = 0.0):
        self.id = type(self).NEURON_ID
        self.type = neuron_type
        self.incoming_edges = [] # TODO: if you use incoming_edges as an argument observe side effects in add_neuron
        self.value = value
        type(self).NEURON_ID += 1
    
    def reset(self):
        self.value = 0.0
    
    def __repr__(self):
        return str(self.__dict__)

In [3]:
# Edge between 2 neurons

import numpy as np

class Gene:
    INNOVATION_NUMBER = 0
    
    def __init__(self, input = 0, output = 0, weight = 0.0, enabled = True):
        self.input = input
        self.output = output
        self.weight = weight
        self.enabled = enabled
        self.innovation = type(self).INNOVATION_NUMBER
        type(self).INNOVATION_NUMBER += 1
    
    def __repr__(self):
        return str(self.__dict__)
    
    def mutate(self, epsilon = 1.0):
        self.weight = np.random.normal(self.weight, epsilon)

In [4]:
from math import exp
import numpy as np
import pprint

# from config.py import *

# The neural network itself, consisting of neurons and edges (a.k.a genes)
class Genome:
    def __init__(self, neurons = [], genes = []):
        self.neurons = neurons
        self.genes = genes # list of edges
        self.fitness = 0
    
    def __repr__(self):
#         return str(self.__dict__)
        return pprint.PrettyPrinter(indent=4).pformat(self.__dict__)
    
    @staticmethod
    def generate():
        input_neurons = [Neuron('input') for _ in range(CONFIG.INPUTS)]
        output_neurons = [Neuron('output') for _ in range(CONFIG.OUTPUTS)]

        neurons = input_neurons + output_neurons
        
        genes = [Gene(input=input_neuron_index, output=output_neuron_index, weight=np.random.randn())
                 for input_neuron_index, _ in enumerate(input_neurons)
                 for output_neuron_index, _ in enumerate(output_neurons)]
        
        for output_neuron_index, output_neuron in enumerate(output_neurons):
            output_neuron.incoming_edges = [gene_index for gene_index, gene in enumerate(genes)
                                            if gene.output == output_neuron_index]

        # TODO: add random weights
            
        return Genome(neurons=neurons, genes=genes)
    
    @staticmethod
    def sigmoid(x):
#         return 2 / (1 + exp(-4.9 * x)) - 1
        return 1 / (1 + exp(-4.9 * x))

    def evaluate_neuron(self, neuron):
        if neuron.value != 0.0:
            return neuron.value
        
        neuron.value = self.sigmoid(sum(
            [self.genes[edge_index].weight * self.evaluate_neuron(self.neurons[self.genes[edge_index].input])
             for edge_index in neuron.incoming_edges
             if self.genes[edge_index].enabled]
        ))
        return neuron.value

    def activate(self, input_values):
        if len(input_values) != CONFIG.INPUTS:
            raise Error("invalid inputs length of {}".format(len(input_values)))
        
        input_index = 0
        for neuron in self.neurons:
            if neuron.type == 'input':
                neuron.value = input_values[input_index]
                input_index += 1
            else:
                neuron.reset()
        
        output_activations = [self.evaluate_neuron(neuron)
                              for neuron in self.neurons
                              if neuron.type == 'output']
        return output_activations
    
    def mutate_weights(self, proportion = CONFIG.MUTATE_WEIGHTS_PROPORTION):
        selected_genes = np.random.choice(self.genes, int(np.floor(len(self.genes) * proportion)))
        for gene in selected_genes:
            gene.mutate()
    
    # Mutation which adds a new neuron on an existing link between 2 other neurons
    def add_neuron(self):
        # Get a random gene (connection)
        gene = np.random.choice(self.genes)
        
        # take its input neuron
        input_neuron = self.neurons[gene.input]
        
        # take its output neuron
        output_neuron = self.neurons[gene.output]
        
        # create a new neuron
        new_neuron_index = len(self.neurons)
        new_neuron = Neuron()
        self.neurons.append(new_neuron)
        
        assert(self.neurons[len(self.neurons) - 1].id == new_neuron.id)
        
        # create a link from input to new
        link1_index = len(self.genes)
        link1 = Gene(input=gene.input, output=new_neuron_index, weight=1)
        self.genes.append(link1)
        
        # create a link from new to output
        link2_index = len(self.genes)
        link2 = Gene(input=new_neuron_index, output=gene.output, weight=gene.weight)
        self.genes.append(link2)
        
        
        print("new_neuron: {}".format(new_neuron))
        print("output_neuron: {}".format(output_neuron))
        
        # connect everything
        new_neuron.incoming_edges = [link1_index]
        
        print("new_neuron: {}".format(new_neuron))
        print("output_neuron: {}".format(output_neuron))
        
        output_neuron.incoming_edges.append(link2_index)
        
        
        print("new_neuron: {}".format(new_neuron))
        print("output_neuron: {}".format(output_neuron))
        
        # disable link from input to output
        gene.enabled = False

In [5]:
class Population:
    def __init__(self):
        self.genomes = [Genome.generate() for _ in range(CONFIG.POPULATION_SIZE)]
    
    def __repr__(self):
        return str(self.__dict__)

In [6]:
population = Population()

genome = population.genomes[0]
genome.activate([np.random.randn() for _ in range(CONFIG.INPUTS)])

[0.9893017937154802, 0.0008203750305685823]

In [7]:
genome.mutate_weights()
genome

{   'fitness': 0,
    'genes': [   {'input': 0, 'output': 0, 'weight': -2.0326784441775385, 'enabled': True, 'innovation': 0},
                 {'input': 0, 'output': 1, 'weight': 1.2586184134103355, 'enabled': True, 'innovation': 1},
                 {'input': 1, 'output': 0, 'weight': -0.42482588960590434, 'enabled': True, 'innovation': 2},
                 {'input': 1, 'output': 1, 'weight': -0.30397649165427026, 'enabled': True, 'innovation': 3},
                 {'input': 2, 'output': 0, 'weight': 0.8679973822889612, 'enabled': True, 'innovation': 4},
                 {'input': 2, 'output': 1, 'weight': -1.6791730455949787, 'enabled': True, 'innovation': 5},
                 {'input': 3, 'output': 0, 'weight': -1.5067444799715797, 'enabled': True, 'innovation': 6},
                 {'input': 3, 'output': 1, 'weight': 1.908663015302331, 'enabled': True, 'innovation': 7}],
    'neurons': [   {'id': 0, 'type': 'input', 'incoming_edges': [], 'value': -0.7786216202358153},
            

In [17]:
genome.add_neuron()

new_neuron: {'id': 12, 'type': 'hidden', 'incoming_edges': [], 'value': 0.0}
output_neuron: {'id': 10, 'type': 'hidden', 'incoming_edges': [16], 'value': 0.0}
new_neuron: {'id': 12, 'type': 'hidden', 'incoming_edges': [20], 'value': 0.0}
output_neuron: {'id': 10, 'type': 'hidden', 'incoming_edges': [16], 'value': 0.0}
new_neuron: {'id': 12, 'type': 'hidden', 'incoming_edges': [20], 'value': 0.0}
output_neuron: {'id': 10, 'type': 'hidden', 'incoming_edges': [16, 21], 'value': 0.0}


In [18]:
genome

{   'fitness': 0,
    'genes': [   {'input': 0, 'output': 0, 'weight': -2.0326784441775385, 'enabled': False, 'innovation': 0},
                 {'input': 0, 'output': 1, 'weight': 1.2586184134103355, 'enabled': False, 'innovation': 1},
                 {'input': 1, 'output': 0, 'weight': -0.42482588960590434, 'enabled': True, 'innovation': 2},
                 {'input': 1, 'output': 1, 'weight': -0.30397649165427026, 'enabled': False, 'innovation': 3},
                 {'input': 2, 'output': 0, 'weight': 0.8679973822889612, 'enabled': True, 'innovation': 4},
                 {'input': 2, 'output': 1, 'weight': -1.6791730455949787, 'enabled': False, 'innovation': 5},
                 {'input': 3, 'output': 0, 'weight': -1.5067444799715797, 'enabled': False, 'innovation': 6},
                 {'input': 3, 'output': 1, 'weight': 1.908663015302331, 'enabled': True, 'innovation': 7},
                 {'input': 3, 'output': 6, 'weight': 1, 'enabled': True, 'innovation': 8},
                

In [10]:
import numpy as np
# n1 = Neuron('hidden', [], 1)
# n1.value

np.random.normal(10, 1)
np.random.choice([1, 2, 3], 2)

array([3, 3])

In [11]:
import gym
import time
env = gym.make('CartPole-v0')

# print(gym.envs.registry.all())

# for episode in range(2000):
    observation = env.reset()
    done = False
    timestamp = 0

    while not done:
        timestamp = timestamp + 1
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        #print("Reward: {}".format(reward))
        if done:
            print("Episode finished after {} timestamps".format(timestamp))
    
env.close()


IndentationError: unexpected indent (<ipython-input-11-af6b93074871>, line 8)